In [1]:
import pandas as pd
import torch
import seaborn as sns
import matplotlib.pylab as plt

In [2]:
df_jsonl = pd.read_json('accepted_2007_to_2018Q4.jsonl.gz', lines=True).set_index('id')

In [3]:
df = pd.read_csv('monteloanco.csv.gz', index_col='id')
df = df.join(df_jsonl, how='inner').reset_index()

In [4]:
df.installment = df.installment.apply(torch.tensor)
df.pymnt = df.pymnt.apply(torch.tensor)

In [5]:
from torch.utils.data import TensorDataset
from torch.nn.utils.rnn import pad_sequence

dataset = TensorDataset(
    torch.tensor(df.index),
    pad_sequence(df.installment, batch_first=True, padding_value=0.), 
    pad_sequence(df.pymnt, batch_first=True, padding_value=0.))

In [14]:
from torch.utils.data import DataLoader
import numpy as np

In [48]:
import pyro
import pyro.distributions as dist
import torch
from pyro.infer import MCMC, NUTS
import numpy as np


def model(batchidx, installments, pymnts):
    """
    Pyro model to infer the transition matrix based on observed state sequences.
    """
    # transpose the input tensors to make stacking/indexing slighly easier
    installments = installments.T #/ self.scaling_factor
    if torch.is_tensor(pymnts): pymnts = pymnts.T #/ self.scaling_factor
        
    # determine the shape of the inputs
    num_timesteps = installments.shape[0]
    batch_size = installments.shape[1]

    # iniitalise other variables
    total_installments = installments.sum(0)
    sim_pymnts = torch.zeros((1, batch_size))
    hidden_states = torch.ones((1, batch_size), dtype=torch.int32)
    
    num_states = 8  # Number of hidden states
    # Prior over the transition matrix: Each row is drawn from a Dirichlet distribution
    transition_matrix = pyro.sample(
        f"transition_matrix_{batchidx}", 
        dist.Dirichlet(torch.ones(batch_size, num_states, num_states))
    )
    
    # Initial state
    with pyro.plate("batch", batch_size, dim=-1):
        for t in range(1, len(installments)):
            # Sample the next state based on the current state's transition probabilities
            new_hidden_states = dist.Categorical(transition_matrix[torch.arange(batch_size), hidden_states]).sample()
    
            # calculate the amount that must have been paid to prompt the status update, where the loan has not been charged off, else 0
            # e.g. a change from 3 month's delinquent up to date implies (3 - 0 + 1)
            new_sim_pymnts = torch.where(
                new_hidden_states < 7,
                (hidden_states - new_hidden_states + 1) * installments[t - 1], # installments is 1 shorter than the simulated vectors as the origin is omitted
                torch.zeros(batch_size)
            )
            
            # overwrite implied payment with the balance where loan has been fully paid
            new_sim_pymnts = torch.where(
                new_hidden_states == 0,
                total_installments - sim_pymnts.sum(0),
                new_sim_pymnts
            )
    
            # Observation model (noisy measurement of hidden state)
            pyro.sample(
                f"pymnt_{batchidx}_{t}", 
                dist.Normal(new_sim_pymnts, 1.),
                obs=pymnts[t]  # Conditioning on the observed state
            )
            #print(hidden_states, new_hidden_states, new_sim_pymnts)
            hidden_states = new_hidden_states

# Set up MCMC to infer the transition matrix
batch_size = 3
nuts_kernel = NUTS(model)
mcmc = MCMC(nuts_kernel, num_samples=batch_size, warmup_steps=100)
for batchidx, (idx, installments, pymnts) in enumerate(DataLoader(dataset, batch_size=batch_size, shuffle=False, num_workers=1)):
    print(installments, pymnts)
    mcmc.run(batchidx, installments, pymnts)
    break

# Extract posterior samples of the transition matrix
posterior_samples = mcmc.get_samples()

# Display inferred transition matrices
print("Posterior Samples of Transition Matrix (first 5 samples):")
print(posterior_samples["transition_matrix_0"][:5])


tensor([[279.6400, 279.6400, 279.6400, 279.6400, 279.6400, 279.6400, 279.6400,
         279.6400, 279.6400, 279.6400, 279.6400, 279.6400, 279.6400, 279.6400,
         279.6400, 279.6400, 279.6400, 279.6400, 279.6400, 279.6400, 279.6400,
         279.6400, 279.6400, 279.6400, 279.6400, 279.6400, 279.6400, 279.6400,
         279.6400, 279.6400, 279.6400,   0.0000,   0.0000,   0.0000,   0.0000,
           0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,
           0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,
           0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,
           0.0000,   0.0000,   0.0000,   0.0000],
        [201.0400, 201.0400, 201.0400, 201.0400, 201.0400, 201.0400, 201.0400,
         201.0400, 201.0400, 201.0400, 201.0400, 201.0400,   0.0000,   0.0000,
           0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,
           0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,
  

Sample: 100%|██| 103/103 [00:02, 40.91it/s, step size=4.58e-17, acc. prob=0.000]


Posterior Samples of Transition Matrix (first 5 samples):
tensor([[[[0.0159, 0.0881, 0.0629, 0.0663, 0.1617, 0.2620, 0.1234, 0.2196],
          [0.0182, 0.1574, 0.0129, 0.0344, 0.1480, 0.3222, 0.2717, 0.0353],
          [0.0122, 0.1500, 0.1195, 0.0146, 0.1115, 0.0249, 0.3126, 0.2546],
          [0.0566, 0.1851, 0.1527, 0.1083, 0.0454, 0.0399, 0.0623, 0.3498],
          [0.0514, 0.0209, 0.2331, 0.0840, 0.2786, 0.2567, 0.0299, 0.0451],
          [0.0136, 0.0963, 0.4439, 0.0403, 0.2462, 0.0179, 0.0306, 0.1111],
          [0.0991, 0.0484, 0.2653, 0.0086, 0.2956, 0.1827, 0.0757, 0.0245],
          [0.2355, 0.0789, 0.0285, 0.0584, 0.0389, 0.3585, 0.0585, 0.1428]],

         [[0.0176, 0.4857, 0.0281, 0.1178, 0.0771, 0.0160, 0.0971, 0.1606],
          [0.0180, 0.1482, 0.1367, 0.0885, 0.2010, 0.1729, 0.1723, 0.0624],
          [0.0244, 0.0729, 0.2203, 0.0603, 0.0619, 0.3785, 0.0280, 0.1537],
          [0.3829, 0.0330, 0.1747, 0.0145, 0.0257, 0.1485, 0.0097, 0.2109],
          [0.1959, 0.0122, 0